In [ ]:
proj_path = '/home/ajhnam/projects/hidden_singles_public/'
save_path = '/data2/pdp/ajhnam/hidden_singles_public/'

In [ ]:
import sys
sys.path.append(proj_path + 'python/')

import random
import numpy as np
import itertools
import pandas as pd
import copy
import os
import glob
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader as DataLoader
from tqdm.auto import tqdm


from hiddensingles.misc import torch_utils as tu
from hiddensingles.misc import pd_utils as pu
from hiddensingles.misc import utils, TensorDict, TensorDictDataset, RRN, DigitRRN, GPUMultiprocessor
from hiddensingles.experiment.sudoku_hs_service import create_tutorial, create_phase1, create_phase2

In [ ]:
device = 0

In [ ]:
def get_results(model, dataset, num_steps=8):
    outputs = model(dataset.inputs, num_steps=num_steps)
    
    goals = tu.expand_along_dim(dataset.goals, 1, num_steps)
    goal_outputs = tu.select(outputs, goals, select_dims=1)
    
    targets = tu.expand_along_dim(dataset.targets, 1, num_steps)
    goal_loss = tu.cross_entropy(goal_outputs, targets)
    goal_probs = tu.select(goal_outputs.softmax(-1), dataset.targets)
    goal_td = TensorDict(loss=goal_loss,
                         probs=goal_probs,
                         outputs=goal_outputs)
    
    coords = tu.expand_along_dim(dataset.coords, 1, num_steps)
    out_exp = tu.expand_along_dim(outputs, 2, 9)
    coord_outputs = tu.select(out_exp, coords, select_dims=1)
    coord_targets = tu.expand_along_dim(dataset.coord_targets, 1, num_steps)
    coord_loss = tu.cross_entropy(coord_outputs, coord_targets)
    coord_probs = tu.select_subtensors(coord_outputs.softmax(-1), coord_targets)
    coord_td = TensorDict(loss=coord_loss,
                          probs=coord_probs,
                          outputs=coord_outputs)

    loss = goal_loss + coord_loss
    correct = dataset.targets == goal_outputs[:,-1].argmax(-1)
    return TensorDict(loss=loss,
                      correct=correct,
                      outputs=outputs,
                      goal=goal_td,
                      coord=coord_td)

In [ ]:
def get_model_performance(model, dataset, df_conditions, model_type, run_id, num_train, epoch):
    # load dataset
    train_dset = dataset.train[:num_train]
    valid_dset = dataset.valid
    test_dset = dataset.test
    
    # get results
    with torch.no_grad():
        train_results = get_results(model, train_dset)
        valid_results = get_results(model, valid_dset)
        test_results = get_results(model, test_dset)

    df_train = train_results[['correct']].to_dataframe({0: 'puzzle_id'})
    df_valid = valid_results[['correct']].to_dataframe({0: 'puzzle_id'})
    df_test = test_results[['correct']].to_dataframe({0: 'puzzle_id'})
    df_train['dataset'] = 'train'
    df_valid['dataset'] = 'valid'
    df_train = pd.concat([df_train, df_valid])
    
    df_train['run_id'] = run_id
    df_train['model'] = model_type
    df_train['num_train'] = num_train
    df_train['epoch'] = epoch
    df_test['run_id'] = run_id
    df_test['model'] = model_type
    df_test['num_train'] = num_train
    df_test['epoch'] = epoch
    
    df_train = df_train[['run_id', 'model', 'num_train', 'epoch', 'dataset', 'puzzle_id', 'correct']]
    df_test = df_test[['run_id', 'model', 'num_train', 'epoch', 'puzzle_id', 'correct']]
    df_test = df_test.merge(df_conditions, on=['run_id', 'puzzle_id'])
    return df_train, df_test

In [ ]:
def get_phase2_conditions(phase2):
    ht = [p.condition.house_type for p in phase2]
    hi = [p.condition.house_index for p in phase2]
    ci = [p.condition.cell_index for p in phase2]
    ds = [p.condition.digit_set for p in phase2]
    conditions = pd.DataFrame(np.array([ht, hi, ci, ds]).T,
                              columns=['house_type', 'house_index', 'cell_index', 'digit_set'])
    return conditions

def hidden_singles_to_tensordict(list_of_hidden_singles, digit_rrn, device='cpu'):
    grids = torch.tensor([a.grid.array for a in list_of_hidden_singles], device=device)
    goals = [p.coordinates['goal'] for p in list_of_hidden_singles]
    goals = torch.tensor([[g.x, g.y] for g in goals], device=device)
    targets = torch.tensor([a.digits['target'] for a in list_of_hidden_singles], device=device) - 1 # make it 0-8
    coords = grids.nonzero()[:,1:].view(len(list_of_hidden_singles), -1, 2)
    coord_targets = tu.select(tu.expand_along_dim(grids, 1, 9), coords) - 1 # make it 0-8
    
    inputs = DigitRRN.make_onehot(grids) if digit_rrn else grids
    return TensorDict(inputs=inputs,
                      grids=grids,
                      goals=goals,
                      targets=targets,
                      coords=coords,
                      coord_targets=coord_targets)

def create_dataset(num_phase1):
    digit_set1 = set(random.sample(set(range(1, 10)), 4))
    digit_set2 = set(random.sample(set(range(1, 10)) - digit_set1, 4))
    tutorial = create_tutorial(digit_set1)
    phase1 = create_phase1(tutorial, num_phase1)
    phase2 = create_phase2(tutorial, digit_set1, digit_set2)
    conditions = get_phase2_conditions(phase2)
    return phase1, phase2, conditions

def train_loop(model, dataloader, optimizer, num_steps=8):
    for dset in dataloader:
        dset = TensorDict(**dset)
        optimizer.zero_grad()
        results = get_results(model, dset, num_steps=num_steps)
        results.loss.backward()
        optimizer.step()

In [ ]:
def run(run_id, num_train, model_type, device):
    assert model_type in ('rrn', 'drrn')
    time_start = datetime.now()
    print("{} Starting run: {}, model: {}, num_train: {}, device: {}".format(
        time_start.strftime("%Y-%m-%d %H:%M:%S"), run_id, model_type, num_train, device), end='\n')
    dirpath = save_path + '{}_hs/tr{}_rid{}/'.format(model_type, num_train, run_id)
    utils.mkdir(dirpath)
    
    if model_type == 'rrn':
        model = RRN(digit_embed_size=10,
                    num_mlp_layers=0,
                    hidden_vector_size=48,
                    message_size=48,
                    encode_coordinates=False).to(device)
        batch_size = 100
    else:
        model = DigitRRN(hidden_vector_size=16,
                         message_size=16).to(device)
        batch_size = 10
    
    num_epochs = 1000
    dataset = TensorDict.load(save_path + 'hs_data.td')
    dataset = dataset[model_type][run_id].to(device)
    dataloader = DataLoader(TensorDictDataset(dataset.train[:num_train]),
                            batch_size=batch_size, shuffle=True)
    df_conditions = pd.read_csv(save_path + 'hs_conditions.tsv', sep='\t')
    df_conditions = df_conditions[df_conditions.run_id == run_id]
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    
    df_train, df_test = get_model_performance(model, dataset, df_conditions,
                                              model_type, run_id, num_train, 0)
    df_train.to_csv(dirpath + 'tr_results_ep_0.tsv', sep='\t', index=False)
    df_test.to_csv(dirpath + 'te_results_ep_0.tsv', sep='\t', index=False)
    for epoch in range(1, num_epochs + 1):
        train_loop(model, dataloader, optimizer)
        if epoch%10 == 0:
            df_train, df_test = get_model_performance(model, dataset, df_conditions, 
                                                      model_type, run_id, num_train, epoch)
            df_train.to_csv(dirpath + 'tr_results_ep_{}.tsv'.format(epoch), sep='\t', index=False)
            df_test.to_csv(dirpath + 'te_results_ep_{}.tsv'.format(epoch), sep='\t', index=False)
            torch.save(model.state_dict(),
                       os.path.join(dirpath, 'epoch_{}.mdl'.format(epoch)))
            
    time_end = datetime.now()
    elapsed = str(time_end - time_start)
    print("{} Completed in {}. run: {}, model: {}, num_train: {}, device: {}".format(
        time_end.strftime("%Y-%m-%d %H:%M:%S"),
        str(elapsed),
        run_id, model_type, num_train, device), end='\n')

# Train Models

Creating dataset and training models should happen only once.

In [ ]:
num_runs = 10
num_train = 500
num_valid = 100
load = True

if load:
    dataset = TensorDict.load(save_path + 'hs_data.td')
    df_conditions = pd.read_csv(save_path + 'hs_conditions.tsv', sep='\t')
else:
    datasets = []
    all_conditions = []
    for i in range(num_runs):
        phase1, phase2, conditions = create_dataset(num_phase1=num_train+num_valid)
        conditions['run_id'] = i
        conditions['puzzle_id'] = range(64)

        rrn_phase1 = hidden_singles_to_tensordict(phase1, digit_rrn=False)
        rrn_phase2 = hidden_singles_to_tensordict([p.hidden_single for p in phase2], digit_rrn=False)
        drrn_phase1 = hidden_singles_to_tensordict(phase1, digit_rrn=True)
        drrn_phase2 = hidden_singles_to_tensordict([p.hidden_single for p in phase2], digit_rrn=True)
        rrn_dset = TensorDict(train=rrn_phase1[:num_train],
                              valid=rrn_phase1[num_train:],
                              test=rrn_phase2)
        drrn_dset = TensorDict(train=drrn_phase1[:num_train],
                               valid=drrn_phase1[num_train:],
                               test=drrn_phase2)
        dset = TensorDict(rrn=rrn_dset, drrn=drrn_dset)
        datasets.append(dset)
        all_conditions.append(conditions)
        
    dataset = TensorDict.stack(datasets, 0)
    dataset.save(save_path + 'hs_data.td')
    df_conditions = pd.concat(all_conditions)
    df_conditions.to_csv(save_path + 'hs_conditions.tsv', sep='\t', index=False)

In [ ]:
df_kwargs = pu.crossing(run_id=range(10),
                        num_train=[25, 50, 100, 300, 500],
                        model_type=['rrn', 'drrn'])
mp = GPUMultiprocessor(run, df_kwargs, devices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
mp.run()

# Metrics

In [ ]:
tr_results = []
te_results = []
for filename in tqdm(glob.glob('/data2/pdp/ajhnam/hidden_singles_public/*_hs/**/tr_results_*.tsv')):
    df = pd.read_csv(filename, sep='\t')
    tr_results.append(df)
for filename in tqdm(glob.glob('/data2/pdp/ajhnam/hidden_singles_public/*_hs/**/te_results_*.tsv')):
    df = pd.read_csv(filename, sep='\t')
    te_results.append(df)
tr_results = pd.concat(tr_results)
te_results = pd.concat(te_results)

In [ ]:
tr_results.to_csv('/data2/pdp/ajhnam/hidden_singles_public/rrn_hs_tr_results.tsv', sep='\t', index=False)
te_results.to_csv('/data2/pdp/ajhnam/hidden_singles_public/rrn_hs_te_results.tsv', sep='\t', index=False)